In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [18]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd

In [19]:
import numpy as np

In [20]:
import time
import matplotlib.pyplot as plt
import math

In [21]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

def SSH_connection(query):
  with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                            passwd=sql_password, db=sql_main_database,
                            port=tunnel.local_bind_port)
    print('connection done')
    df_result = pd.read_sql_query(query,conn)
  
  return df_result

In [22]:
# choose users with the same firstLogin day -- e.g. all users who first logged in on 2016-04-01
# choose a timeframe for observation (what is a good range? why? | 7 months-1 year | LONGER IS BETTER) -- e.g. from 2016-04-01 to 2017-04-01 | is this enough time to get fair results?
# observe where users perform their first and last recommender change wrt the chosen time period
# observe intermediary actions on a monthly basis -- actions summary for each month/ each quarter during the observation period | per quarter | per session makes the most sense
# this analysis can help answer 'what do users experiment with towards the beginning of their use of MovieLens versus 7 months into using the platform?'
# 'are there noticeable patterns in actions leading to a recommender change?'
# can be extended to different time periods and different firstLogin days | results can be compared for such time periods

In [23]:
# function to determine when an action was first performed by a user
# df is the dataframe with all actions of all users; action is the action of interest; 
# fields is a tuple of the fields in the df that correspond to the desired output information
# returns a dictionary where key=userId, val=time at which action was performed (in the current context)
def first_occurence(df, action, fields):
    change_tstamps = {}

    key = fields[0]
    count_of = fields[1]
    val = fields[2]
    rows = df.shape[0]
    
    for k in range(rows):
        userId = df.loc[k].at[key]

        # checking if the row contains the action of interest and if this is the first occurence of that user-action combination
        if df.loc[k].at[count_of]==action and userId not in change_tstamps.keys():
            change_tstamps[userId] = df.loc[k].at[val] # inserting appropriate values into the dictionary
    
    return change_tstamps


In [24]:
from datetime import datetime, timedelta

In [25]:
# function remove_rows
# takes a df, an integer representing the desired difference, and the fields of interest
# eliminates those rows in the df where the difference between the two fields of interest are lower than the desired difference
# does not modify the original df
# returns the count of rows in the modified df
def remove_rows_opt(dfs, desired_diff, fields):
    alt_df = dfs[0]
    df_to_compare = dfs[1]
    rows = alt_df.shape[0]

    for k in range(rows):
        alt_df_userId = alt_df.loc[k].at[fields[0]]
        row = df_to_compare.loc[df_to_compare[fields[1]] == alt_df_userId]
        index = row.index[0]
        diff = row.loc[index, fields[2]]
        if diff<desired_diff:
            alt_df = alt_df.drop(k, 0)

        # if 
        #     alt_df = alt_df.drop(k, 0)
        # else:
        #     print("diff: ", diff)
        #     print("days:", diff.days)
    return alt_df.shape[0]

In [26]:
def remove_rows(df, desired_diff):
    alt_df = df
    rows = alt_df.shape[0]
    for k in range(rows):
        firstLogin = alt_df.loc[k].at['firstLogin']
        lastLogin = alt_df.loc[k].at['lastLogin']
        diff = lastLogin - firstLogin

        if diff.days>desired_diff:
            alt_df = alt_df.drop(k, 0)

    return alt_df

In [27]:
query = ''' SELECT userId, firstLogin, lastLogin, tstamp from ML3_mirror.user_data WHERE EXISTS
            (SELECT userId from ML3_mirror.log_action WHERE user_data.userId=log_action.userId AND tstamp<'2021-09-02' AND tstamp>'2018-09-01')
            AND firstLogin<'2019-09-02' AND firstLogin>='2018-09-01' '''

# query = ''' SELECT userId, firstLogin, lastLogin, tstamp from ML3_mirror.user_data WHERE EXISTS
#             (SELECT userId from ML3_mirror.log_action WHERE tstamp<'2021-09-02' AND tstamp>='2018-09-01')
#             AND firstLogin<'2019-09-02' AND firstLogin>='2018-09-01' '''
first_Logins = SSH_connection(query)
print("None removed: ", first_Logins.shape[0])
# first_Logins.head()

# login_at = first_Logins.loc[first_Logins['userId'] == 374395]
# index = login_at.index[0]
# print(login_at.loc[index, 'tstamp'])

print("Removing if diff<180: ", remove_rows(first_Logins, 180).shape[0])

#consider a count of sessions
#user_login table -- could use sessioning info
#bringing down sample size? -- considering only a portion of users 

#16232 | 3374 | 2747 | 2341 | 1854

connection done
None removed:  16116
Removing if diff<180:  13778


In [28]:
print(first_Logins.shape[0])
first_Logins.head()

16116


,userId,firstLogin,lastLogin,tstamp
0,326572,2018-09-01,2018-09-01,2018-09-01 00:02:56
1,326574,2018-09-01,2018-09-01,2018-09-01 01:27:34
2,326576,2018-09-01,2018-09-01,2018-09-01 03:35:43
3,326577,2018-09-01,2018-09-01,2018-09-01 05:14:13
4,326583,2018-09-01,2018-09-01,2018-09-01 08:14:30


In [29]:
# df containing users who DID NOT use the system for the time period of interest AND are within the desired firstLogin range
users_sample = remove_rows(first_Logins, 180)
print("After drop: ", users_sample.shape[0])

After drop:  13778


In [ ]:
# query = ''' SELECT userId, action, tstamp from ML3_mirror.log_action
#             WHERE log_action.tstamp<'2021-09-02' AND log_action.tstamp>'2018-09-01' '''

query = ''' SELECT userId, loginId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2019-09-02' AND user_data.firstLogin>='2018-09-01'
            AND log_action.userId = user_data.userId)
            AND log_action.tstamp<'2021-09-02' AND log_action.tstamp>'2018-09-01' '''
result = SSH_connection(query)
print("done")
#4m 48.9s

In [31]:
query = ''' SELECT userId, action, MIN(tstamp) from ML3_mirror.log_action WHERE EXISTS
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2019-09-02' AND user_data.firstLogin>='2018-09-01'
            AND log_action.userId = user_data.userId)
            AND action = 'recommender-change' AND tstamp<'2021-09-02' AND tstamp>'2018-09-01' GROUP BY userId '''
first_rec_change = SSH_connection(query)
print("done")
print(first_rec_change.shape[0])

connection done
done
14866


In [32]:
query = ''' SELECT userId, MIN(tstamp) from ML3_mirror.user_login WHERE tstamp<'2019-09-02' and tstamp>='2018-09-01' GROUP BY userId'''
firstLogin_tstamps = SSH_connection(query)
print("done")
print(firstLogin_tstamps.shape[0])
print(min(firstLogin_tstamps['MIN(tstamp)']))
print(max(firstLogin_tstamps['MIN(tstamp)']))

connection done
done
24150
2018-09-01 00:00:01
2019-09-01 23:51:36


In [33]:
indices = list(users_sample.index)
length = len(indices)
frc_modified = first_rec_change
for i in range(length):
    row_index = indices[i]
    userId = users_sample.loc[row_index].at['userId']
    frc_modified = frc_modified[frc_modified['userId'] != userId]

print(frc_modified.shape[0])
# frc_modified.head()

indices = list(frc_modified.index)
length = len(indices)
diffs_dict = {}
for j in range(length):
    userId = frc_modified.loc[indices[j]].at['userId']
    flogin_row = firstLogin_tstamps.loc[firstLogin_tstamps['userId'] == userId]
    flogin_tstamp = flogin_row.loc[flogin_row.index[0], 'MIN(tstamp)']
    diffs_dict[userId] = frc_modified.loc[indices[j]].at['MIN(tstamp)'] - flogin_tstamp

n = 0
for key,val in diffs_dict.items():
    if n<50:
        print(key,val)
        n += 1
    else:
        break

print(len(diffs_dict))
print(min(diffs_dict.values()))
print(max(diffs_dict.values()))

2298
326581 0 days 00:01:10
326592 0 days 00:00:46
326596 0 days 00:02:13
326614 0 days 00:01:07
326615 0 days 05:58:58
326632 0 days 00:01:02
326636 0 days 00:03:45
326637 0 days 00:00:41
326639 0 days 00:00:53
326645 0 days 00:03:41
326648 0 days 00:01:29
326650 0 days 00:01:36
326652 0 days 00:00:47
326663 0 days 00:00:37
326664 0 days 00:00:47
326672 0 days 00:00:35
326681 0 days 00:02:27
326709 0 days 00:00:30
326711 0 days 00:02:19
326719 0 days 00:04:05
326723 0 days 00:00:54
326724 0 days 00:00:46
326727 0 days 00:12:14
326735 0 days 00:00:48
326736 0 days 00:00:52
326758 0 days 00:00:46
326765 0 days 00:00:20
326771 0 days 00:00:47
326775 0 days 00:01:04
326776 0 days 00:01:06
326782 0 days 00:00:56
326784 0 days 00:00:47
326789 0 days 00:03:17
326792 0 days 00:00:51
326794 0 days 00:00:48
326796 0 days 00:00:49
326805 0 days 00:01:12
326810 0 days 00:00:31
326823 0 days 00:02:28
326825 0 days 00:04:06
326833 0 days 00:00:54
326839 0 days 00:01:34
326841 0 days 00:00:31
326846

In [ ]:
# rows = users_sample.shape[0]
indices = list(users_sample.index)
length = len(indices)
for i in range(length):
    row_index = indices[i]
    userId = users_sample.loc[row_index].at['userId']
    result = result[result['userId'] != userId]

print("number of user entries available: ", result.shape[0])
#48m 23.4s | STILL NOT DONE AHHH

# explore dataframe filter operations
# joins in queries
# set logic 
# having filter


In [9]:
start_time = time.time()

query = ''' SELECT userId, firstLogin, lastLogin, tstamp from ML3_mirror.user_data WHERE EXISTS
            (SELECT userId from ML3_mirror.log_action WHERE tstamp<'2021-09-02' AND tstamp>='2018-09-01' AND user_data.userId=log_action.userId)
            AND firstLogin<'2019-09-02' AND firstLogin>='2018-09-01' '''

first_Logins = SSH_connection(query)
# print("Removing if diff<30: ", remove_rows(first_Logins, 30))
# print("Removing if diff<90: ", remove_rows(first_Logins, 90))
print("Removing if diff<180: ", remove_rows(first_Logins, 180))

query = ''' SELECT userId, loginId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2019-09-02' AND user_data.firstLogin>='2018-09-01'
            AND log_action.userId = user_data.userId)
            AND log_action.tstamp<'2021-09-02' AND log_action.tstamp>'2018-09-01' '''

            
users_df = SSH_connection(query)
print(users_df.shape[0])
users_df.head()

users_df_sorted = users_df.sort_values(by=['userId', 'tstamp'])
print(users_df_sorted.shape[0])
users_df_sorted.head()
# rows = users_df.shape[0]
# for k in range(rows):
#     users_df_userId = users_df.loc[k].at['userId']
#     if users_df_userId not in first_Logins['userId']:
#         users_df = users_df.drop(k, 0)

print("remove_rows: ", time.time() - start_time)

#16m 24.9s still incomplete | done till line 17, loop takes time
#4889455 records

connection done
Removing if diff<180:         userId  firstLogin   lastLogin              tstamp
0      326572  2018-09-01  2018-09-01 2018-09-01 00:02:56
1      326574  2018-09-01  2018-09-01 2018-09-01 01:27:34
2      326576  2018-09-01  2018-09-01 2018-09-01 03:35:43
3      326577  2018-09-01  2018-09-01 2018-09-01 05:14:13
4      326583  2018-09-01  2018-09-01 2018-09-01 08:14:30
...       ...         ...         ...                 ...
16104  343164  2019-09-01  2019-09-12 2019-09-12 03:18:49
16105  343138  2019-09-01  2019-09-14 2019-09-14 11:16:35
16106  343142  2019-09-01  2019-09-17 2019-09-17 11:01:02
16107  343160  2019-09-01  2019-11-16 2019-11-16 00:37:16
16108  343159  2019-09-01  2019-11-29 2019-11-29 18:41:56

[13778 rows x 4 columns]
connection done
4889455
4889455
remove_rows:  238.68573188781738


In [12]:
users_df_sorted.head(10)

,userId,loginId,action,tstamp
37,326573,JXZm7NC,pageview,2018-09-01 01:19:52
38,326573,JXZm7NC,recommender-change,2018-09-01 01:20:25
39,326573,JXZm7NC,pageview,2018-09-01 01:20:26
40,326573,JXZm7NC,pageview,2018-09-01 01:20:55
41,326573,JXZm7NC,pageview,2018-09-01 01:21:15
42,326573,JXZm7NC,pageview,2018-09-01 01:21:44
43,326573,JXZm7NC,rating,2018-09-01 01:21:48
84,326573,JXZm7NC,pageview,2018-09-01 01:48:30
133,326573,JXZm7NC,rating,2018-09-01 01:53:15
145,326573,JXZm7NC,pageview,2018-09-01 01:53:35


In [14]:
query = ''' SELECT userId, loginId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2019-09-02' AND user_data.firstLogin>='2018-09-01'
            AND log_action.userId = user_data.userId)
            AND log_action.tstamp<'2021-09-02' AND log_action.tstamp>'2018-09-01' ORDER BY log_action.userId, log_action.tstamp '''
users_df_new = SSH_connection(query)
print(users_df_new.shape[0])
users_df_new.head()

connection done
4889455


,userId,loginId,action,tstamp
0,326572,f69CROC,pageview,2018-09-01 00:02:57
1,326572,f69CROC,recommender-change,2018-09-01 00:03:44
2,326572,f69CROC,pageview,2018-09-01 00:03:44
3,326572,f69CROC,pageview,2018-09-01 00:03:53
4,326572,f69CROC,rating,2018-09-01 00:04:02


In [ ]:
rows = users_df_sorted.shape[0]
for k in range(rows):
    users_df_userId = users_df_new.loc[k].at['userId']
    if users_df_userId not in first_Logins['userId']:
        users_df_sorted = users_df_new[users_df_new['userId'] != users_df_userId]
print(users_df_new.shape[0])

In [ ]:
query = ''' SELECT userId, firstLogin, lastLogin, DATEDIFF(lastLogin, firstLogin) as diff from ML3_mirror.user_data WHERE EXISTS
            (SELECT userId from ML3_mirror.log_action WHERE tstamp<'2018-11-01' AND tstamp>='2018-09-01' AND user_data.userId=log_action.userId)
            AND firstLogin<'2018-11-01' AND firstLogin>='2018-09-01' '''

first_Logins = SSH_connection(query)
# print("Removing if diff<30: ", remove_rows(first_Logins, 30))
# print("Removing if diff<90: ", remove_rows(first_Logins, 90))
# print("Removing if diff<180: ", remove_rows(first_Logins, 180))

query = ''' SELECT userId, loginId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2018-09-31' AND user_data.firstLogin>='2018-09-01'
            AND log_action.userId = user_data.userId)
            AND log_action.tstamp<'2018-11-01' AND log_action.tstamp>'2018-09-01' '''

users_df = SSH_connection(query)
print(users_df.shape[0])

rows = remove_rows_opt((users_df, first_Logins), 180, ('userId', 'userId', 'diff'))

In [ ]:
#users who logged in within 2021-06-01 - 2021-08-31
#2899

#3120
#3119
# and firstLogin is NOT NULL
# AND userId>0 
# AND isDeleted=0
# AND isAdmin=0
#

# (SELECT userId, loginName, firstLogin, lastLogin, tstamp from ML3_mirror.user_data 
#              WHERE firstLogin<'2021-09-01' AND firstLogin>'2021-06-01') 
query = ''' SELECT userId, loginId, action, tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.user_data WHERE user_data.firstLogin<'2021-09-01' AND user_data.firstLogin>'2021-05-31'
            AND log_action.userId = user_data.userId)
            AND log_action.tstamp<'2021-09-01' AND log_action.tstamp>'2021-05-31' '''
users_df = SSH_connection(query)

# key=userId, val=tstamp at which first recommender-change occured
tstamps_dict = first_occurence(users_df, 'recommender-change', ('userId', 'action', 'tstamp'))
# first_change = tstamps_dict[374395]
# print(first_change)

rows = first_Logins.shape[0]
print(type(tstamps_dict))

# SELECT MIN(tstamp) ... GROUP BY userId
# date -- day without a time | datetime - an instantaneous point in time | timedelta -- count of seconds between two tstamps
# looping through the dictionart to calculate the time difference between the first recommender change and the first login time
# this assumed that the tstamp column in the user_data table corresponds to the firstLogin time, but this isn't correct
for userId,tstamp in tstamps_dict.items():
    
    login_at_df = first_Logins.loc[first_Logins['userId'] == userId]
    index = login_at_df.index[0]
    login_at = login_at_df.loc[index, 'tstamp']
    diff = tstamp - login_at
    tstamps_dict[userId] = diff.total_seconds()
# login_at = first_Logins.loc[0,'tstamp']
# print(login_at)

# for i in range(rows):
#     userId = first_Logins.loc[i].at['userId']
#     change_at = tstamps_dict[userId]
#     diff = change_at - first_Logins.loc[i].at['tstamp']
#     tstamps_dict[userId] = diff.total_seconds()

# print(tstamps_dict)
k=0
for key,val in tstamps_dict.items():
    if k <15:
        print(key, val)
    else:
        break
    k += 1

# print(users_df)
